In [1]:
import torch
import os
from ultralytics import YOLO
from ultralytics.models.yolo.model import DetectionModel # Ensure this import is correct

In [2]:
# --- 1. Configuration ---
CONFIG_NAME = 'yolo11m-nn-ds.yaml'  # Configuration file name
# File Paths
current_dir = os.getcwd()
target_yaml_path = os.path.join(current_dir, 'ultralytics', 'cfg', 'models', '11', CONFIG_NAME)
source_weights_path = os.path.join(current_dir, 'yolo11m.pt')           # Pre-trained standard YOLOv11s
output_weights_path = os.path.join(current_dir, 'runs', 'models', f'{CONFIG_NAME.split(".")[0]}.pt') # Output path for transferred weights

# Data Paths
data_path = os.path.join(os.getcwd(), 'datasets', 'split', '4', 'data.yaml')
test_data_path = os.path.join(os.getcwd(), 'datasets', 'split', '4', 'data.yaml')

# Model Parameters (Ensure these match your dataset and model)
nc = 1 # Number of classes (adjust if your dataset differs from COCO)

# LCBHAM Layer Indices (Confirm these match your target_yaml_path)
lcbham_layers_indices = {17}

In [ ]:
# --- 2. Perform Weight Transfer (if necessary) ---

if not os.path.exists(output_weights_path):
    print(f"Transfer weights file '{output_weights_path}' not found. Performing weight transfer...")

    # --- Load Source Weights ---
    print(f"Loading source weights from {source_weights_path}...")
    try:
        source_ckpt = torch.load(source_weights_path, map_location=torch.device('cpu'), weights_only=False)

        if hasattr(source_ckpt.get('model'), 'state_dict'):
            source_state_dict = source_ckpt['model'].float().state_dict()
        elif isinstance(source_ckpt.get('model'), dict):
             source_state_dict = source_ckpt['model']
        elif isinstance(source_ckpt, dict) and not 'model' in source_ckpt:
             source_state_dict = source_ckpt
        else:
            print("Available keys in source_ckpt:", source_ckpt.keys() if isinstance(source_ckpt, dict) else "Not a dict")
            raise ValueError("Could not extract state_dict from source checkpoint.")
        print(f"Source weights loaded and state_dict extracted.")
    except Exception as e:
        print(f"Error loading source weights: {e}")
        exit()

    # --- Build Target Model Structure ---
    print(f"Building target model structure from {target_yaml_path}...")
    try:
        # Ensure your custom LCBHAM module is defined/imported before this line
        target_model = DetectionModel(cfg=str(target_yaml_path), ch=3, nc=nc)
    except Exception as e:
         print(f"Error building target model: {e}")
         print("Ensure LCBHAM is defined, the YAML path is correct, and nc={nc} is appropriate.")
         exit()
    target_state_dict = target_model.state_dict()
    print(f"Target model structure built.")

    # --- Weight Transfer Logic ---
    print("Starting weight transfer...")
    new_state_dict = {}
    transferred_count = 0
    skipped_count = 0
    lcbham_conv_transferred = set()
    lcbham_bn_transferred = set()

    for k_target, v_target in target_state_dict.items():
        k_source = k_target
        layer_index_str = k_target.split('.')[1] # Get layer index string like '17'

        if layer_index_str.isdigit() and int(layer_index_str) in lcbham_layers_indices:
            layer_index = int(layer_index_str)
            if f".{layer_index_str}.conv_block.0." in k_target: # Conv part of LCBHAM
                k_source = k_target.replace("conv_block.0.", "conv.")
                lcbham_conv_transferred.add(layer_index)
            elif f".{layer_index_str}.conv_block.1." in k_target: # BN part of LCBHAM
                k_source = k_target.replace("conv_block.1.", "bn.")
                lcbham_bn_transferred.add(layer_index)

        if k_source in source_state_dict and source_state_dict[k_source].shape == v_target.shape:
            new_state_dict[k_target] = source_state_dict[k_source]
            transferred_count += 1
        else:
            new_state_dict[k_target] = v_target
            skipped_count += 1

    print("\n--- Transfer Summary ---")
    print(f"Total keys in target model: {len(target_state_dict)}")
    print(f"Weights transferred: {transferred_count}")
    print(f"Weights skipped/kept from target: {skipped_count}")
    for idx in lcbham_layers_indices:
        if idx in lcbham_conv_transferred: print(f"Successfully mapped Conv weights for LCBHAM layer {idx}.")
        if idx in lcbham_bn_transferred: print(f"Successfully mapped BN weights for LCBHAM layer {idx}.")

    # --- Load New State Dict and Save Checkpoint ---
    target_model.load_state_dict(new_state_dict, strict=False)
    print("\nLoaded transferred weights into target model structure.")

    output_ckpt = {
        'epoch': -1,
        'best_fitness': None,
        'model': target_model,
        'ema': None,
        'updates': None,
        'train_args': {}, # Use empty dict for compatibility
        'date': None
}

    # Get the directory part of the output path using os.path
    output_directory = os.path.dirname(output_weights_path)

    # Create the directory recursively, ignoring errors if it exists
    os.makedirs(output_directory, exist_ok=True)
    # --- End Replacement Code ---

    # Save the checkpoint (this line remains unchanged)
    torch.save(output_ckpt, output_weights_path)
    print(f"Saved model with transferred weights to {output_weights_path}")
    print("\nWeight transfer complete.")

else:
    print(f"Found existing transfer weights file: '{output_weights_path}'. Skipping transfer step.")

In [3]:
# --- 3. Fine-Tuning ---
print("\n--- Starting Fine-Tuning ---")

# Load the model with transferred weights
model = YOLO('yolo11m.yaml') # Load the result of the transfer

# --- Train Model ---
results = model.train(
    # ** Essential Paths & Config **
    data=str(data_path),                # Path to your dataset YAML file
    name='yolo11m-yaml',                 # Name for the training run directory
    exist_ok=False,                # Error if run name already exists
    save=True,                     # Save checkpoints and final model
    pretrained=False,
    
    # ** Requested Hyperparameters **
    epochs=50,                    # Number of training epochs
    batch=12,                       # Batch size
    imgsz=640,                     # Input image size (height=width=640)

    # ** Data Augmentation Control (ONLY MOSAIC ENABLED) **
    augment=False,                  # MUST be True to enable the augmentation pipeline for mosaic
)


--- Starting Fine-Tuning ---
New https://pypi.org/project/ultralytics/8.3.127 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.yaml, data=/home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/data.yaml, epochs=50, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo11m-yaml, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, 

train: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/train/labels.cache... 9450 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9450/9450 [00:00<?, ?it/s]
val: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/valid/labels.cache... 1350 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1350/1350 [00:00<?, ?it/s]


Plotting labels to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.00046875), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      6.05G      2.536      1.745      1.525        176        640: 100%|██████████| 788/788 [02:12<00:00,  5.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:04<00:00, 11.47it/s]


                   all       1350      23084      0.838      0.591      0.639      0.286

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      6.32G     0.8192     0.3722     0.8358        101        640: 100%|██████████| 788/788 [02:08<00:00,  6.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:04<00:00, 11.57it/s]


                   all       1350      23084      0.823      0.752      0.762      0.446

50 epochs completed in 1.888 hours.
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml/weights/last.pt, 40.5MB
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml/weights/best.pt, 40.5MB

Validating /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)
YOLO11m summary (fused): 125 layers, 20,033,116 parameters, 0 gradients, 67.663 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:06<00:00,  8.99it/s]


                   all       1350      23084      0.819      0.743      0.761      0.451
                player       1350      19280      0.924      0.962      0.971      0.629
            goalkeeper        760        760       0.97      0.933      0.974      0.551
               referee       1191       1927      0.829      0.941      0.942      0.574
                  ball       1117       1117      0.552      0.135      0.157     0.0494
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11m-yaml


In [ ]:
# --- 4. Validation ---
print("\n--- Starting Validation ---")

print(f"Validating on Game ID 4: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (Game ID 4):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")

In [ ]:
model = YOLO('./runs/detect/yolo11m-nn-ds/weights/best.pt')

print("\n--- Starting Validation ---")

print(f"Validating on Game ID 4: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (Game ID 4):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")

In [ ]:
model = YOLO('./runs/detect/yolo11m-nn-ds/weights/best.pt')

print("\n--- Starting Validation ---")

print(f"Validating on Game ID 4: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (Game ID 4):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")